### Setup

In [1]:
""" Python Standard Library """
import os
import datetime as dt
import json
import logging
""" Third Party Imports """
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
""" Local Imports """
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
print(os.getcwd())
from covid_model.model import CovidModel
from covid_model.runnable_functions import do_single_fit, do_fit_scenarios, do_create_multiple_reports
from covid_model.utils import setup, get_filepath_prefix
from covid_model.analysis.charts import plot_transmission_control

os.environ['gcp_project'] = 'co-covid-models'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "co-covid-models-credentials.json"

/Users/emwu9912/PycharmProjects/covid-models


In [2]:
# set up the output directory for this Jupyter notebook
outdir = setup("20221004_scenario_projections.ipynb")

2022/10/06 11:30:04 INFO|============================================================


### Fit an initial scenario through February 2022

In [ ]:
# designate the arguments for how the model will behave
model_args = {
    'params_defs': 'covid_model/analysis/20221004_oct_gov_briefing/20221004_scenario_params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/analysis/20221004_oct_gov_briefing/20221004_vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2024-01-01',
}

# this is how the fit will behave
# place the outdir argument here to tell the model fit where to go
fit_args = {'outdir': outdir,
            'fit_end_date': '2022-02-28'
}

# because all the scenarios are the same 
model = do_single_fit(**fit_args, **model_args)

### Create and run scenarios from Feb 2022 to present

In [8]:
multiprocess = 4

scenario_params = json.load(open("covid_model/analysis/20221004_oct_gov_briefing/20221004_scenario_params.json"))

model_args = {
    'base_spec_id': 4167 #model.spec_id, # use the spec id that was output from the model fit
}
model_fit_args = {
    'outdir': outdir,
    'fit_start_date': '2022-03-01', # set the start date for the earliest point at which the scenarios start to differ from one another
    'pre_solve_model': True # force the model to establish initial conditions so the fit can start on the fit start date
}

# define vaccine effectiveness for < 5 (this is a multiplier for the baseline vaccine effectiveness for 0-19)
vacc_eff_lt5 = 0.5

# Create different scenarios to model
scenario_model_args = []
for vx_seed in [0, 5]:
    for vir_mult in [0.833, 2.38]:
        hrf = {"2020-01-01": 1, "2022-03-01": (0.66 + 0.34*0.8),
               "2022-03-15": (0.34 + 0.66*0.8), "2022-03-30": 0.8}
        vx_adjust = [{"param": "vx_seed",
                      "vals": {"2020-01-01": 0, "2022-09-30": vx_seed, "2022-10-30": 0},
                      "desc": "Variant X seeding"}]
        vir_adjust = [{"param": "hosp",
                       "attrs": {"variant": "vx"},
                       "mults": {"2020-01-01": vir_mult},
                       "desc": "Variant X hospitalization multiplier"}]
        lt5_vacc_adjust = [{"param": "immunity",
                            "attrs": {'age': '0-19', 'vacc': 'shot1'},
                            "mults": {"2020-01-01": 1,
                                      "2022-06-20": 0.99 + 0.01*vacc_eff_lt5,
                                      "2022-06-30": 0.98 + 0.02*vacc_eff_lt5,
                                      "2022-07-10": 0.97 + 0.03*vacc_eff_lt5,
                                      "2022-07-20": 0.96 + 0.04*vacc_eff_lt5,
                                      "2022-08-10": 0.95 + 0.05*vacc_eff_lt5,
                                      "2022-08-30": 0.94 + 0.06*vacc_eff_lt5,
                                      "2022-09-20": 0.93 + 0.07*vacc_eff_lt5},
                            "desc": "weighted average using share of 0-19 getting shot1 who are under 5"}]
        scenario_model_args.append({'params_defs': scenario_params + vx_adjust + vir_adjust + lt5_vacc_adjust,
                                    'hosp_reporting_frac': hrf,
                                    'tags': {'vx_seed': vx_seed,
                                             'vir_mult': vir_mult,
                                             'booster_mult': 0}})
            

In [9]:
# check how many scenarios there are
len(scenario_model_args)

4

In [10]:
# run the scenarios
models = do_fit_scenarios(base_model_args=model_args, scenario_args_list=scenario_model_args, fit_args=model_fit_args, multiprocess=multiprocess)

2022/10/06 11:50:12 INFO|============================================================
2022/10/06 11:50:12 INFO|============================================================
2022/10/06 11:50:12 INFO|============================================================
2022/10/06 11:50:12 INFO|============================================================
2022/10/06 11:50:15 INFO|-----------|{'vx_seed': 5, 'vir_mult': 0.833, 'booster_mult': 0} Retrieving vaccinations data
2022/10/06 11:50:15 INFO|-----------|{'vx_seed': 5, 'vir_mult': 2.38, 'booster_mult': 0} Retrieving vaccinations data
2022/10/06 11:50:15 INFO|-----------|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 0} Retrieving vaccinations data
2022/10/06 11:50:15 INFO|-----------|{'vx_seed': 0, 'vir_mult': 0.833, 'booster_mult': 0} Retrieving vaccinations data
2022/10/06 11:50:22 INFO|-----------|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 0} Constructing vaccination projections
2022/10/06 11:50:22 INFO|-----------|{'vx_seed': 5, 'vir

2022/10/06 11:52:34 INFO|--------|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '2'}: Transmission control fit 3/5 completed in 12.963229500000011 seconds: {823: {'co': 0.8484155029800612}, 837: {'co': 0.8463117345010214}, 851: {'co': 0.8328439858664983}, 865: {'co': 0.8617811255822962}, 879: {'co': 0.8534447993709878}, 893: {'co': 0.8576382092966356}}
2022/10/06 11:52:34 INFO|---------|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '2'}: Running forward sim
2022/10/06 11:52:34 INFO|--------|{'vx_seed': 5, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '2'}: Transmission control fit 3/5 completed in 12.884083207999993 seconds: {823: {'co': 0.8484155029800612}, 837: {'co': 0.8463117345010214}, 851: {'co': 0.8328439858664983}, 865: {'co': 0.8617811255822962}, 879: {'co': 0.8534447993709878}, 893: {'co': 0.8576382092966356}}
2022/10/06 11:52:34 INFO|---------|{'vx_seed': 5, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '2'}: Running forward sim


2022/10/06 11:53:11 INFO|--------|{'vx_seed': 5, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '4', 'run_type': 'fit'}: fitted TC: {0: {'co': 0.8587936394045163}, 14: {'co': 0.6897761516173341}, 28: {'co': 0.5461822485713237}, 42: {'co': 1.47725949066134e-14}, 56: {'co': 0.6964649573507541}, 70: {'co': 0.8333063430025454}, 84: {'co': 0.8295949481128148}, 98: {'co': 0.8317180713022121}, 112: {'co': 0.8774454578075962}, 126: {'co': 0.8243715631798311}, 140: {'co': 0.7604768954200014}, 154: {'co': 0.6712541486338183}, 168: {'co': 0.8430274765831426}, 182: {'co': 0.7826504004227312}, 196: {'co': 0.8191652899986039}, 210: {'co': 0.7726023562796098}, 224: {'co': 0.7713430225629265}, 238: {'co': 0.7185520652914471}, 252: {'co': 0.7221395048128676}, 266: {'co': 0.6850199045128585}, 280: {'co': 0.7198410210178658}, 294: {'co': 0.7578493583598209}, 308: {'co': 0.8225928618348625}, 322: {'co': 0.8115169202432801}, 336: {'co': 0.7825636926828162}, 350: {'co': 0.7953853340104805}, 364: {'co': 0

### Run the report for each fit model

In [13]:
# here you can also specify which variants you want to calculate immunity for
do_create_multiple_reports(models, multiprocess=multiprocess, outdir=outdir, prep_model=False, solve_model=True, immun_variants=['ba45', 'vx'], from_date='2022-01-01')

2022/10/10 10:52:41 INFO|============================================================
2022/10/10 10:52:41 INFO|--------|Solving model
2022/10/10 10:52:41 INFO|============================================================
2022/10/10 10:52:41 INFO|--------|Solving model
2022/10/10 10:52:42 INFO|============================================================
2022/10/10 10:52:42 INFO|--------|Solving model
2022/10/10 10:52:43 INFO|============================================================
2022/10/10 10:52:43 INFO|--------|Solving model
2022/10/10 10:52:45 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/10/10 10:52:45 WARNING|No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
2022/10/10 10:52:46 WARNING|No artists with labels found to put in legend.  Note that artists 

In [12]:
logging.info('Projecting')
for model in models:
    logging.info('')
    #model.prep()  # don't think we need to prep anymore.
    model.solve_seir()

    model.solution_sum_df(['seir', 'variant', 'immun']).unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_seir_variant_immun_total_all_at_once_projection.csv')
    model.solution_sum_df().unstack().to_csv(get_filepath_prefix(outdir, tags=model.tags) + 'states_full_projection.csv')

    logging.info(f'{str(model.tags)}: Running forward sim')
    fig = plt.figure(figsize=(10, 10), dpi=300)
    ax = fig.add_subplot(211)
    hosps_df = model.modeled_vs_observed_hosps().reset_index('region').drop(columns='region')
    hosps_df.plot(ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    ax = fig.add_subplot(212)
    plot_transmission_control(model, ax=ax)
    ax.set_xlim(dt.datetime.strptime('2022-01-01', "%Y-%m-%d").date(), dt.datetime.strptime('2024-01-01', "%Y-%m-%d").date())
    plt.savefig(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast.png')
    plt.close()
    hosps_df.to_csv(get_filepath_prefix(outdir, tags=model.tags) + '_model_forecast.csv')
    json.dump(model.tc, open(get_filepath_prefix(outdir, tags=model.tags) + 'model_forecast_tc.json', 'w'))

logging.info('Running reports')

2022/10/06 12:00:15 INFO|Projecting
2022/10/06 12:00:15 INFO|
2022/10/06 12:00:35 INFO|{'vx_seed': 0, 'vir_mult': 0.833, 'booster_mult': 0, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/10/06 12:00:37 INFO|
2022/10/06 12:00:56 INFO|{'vx_seed': 0, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/10/06 12:00:57 INFO|
2022/10/06 12:01:16 INFO|{'vx_seed': 5, 'vir_mult': 0.833, 'booster_mult': 0, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/10/06 12:01:18 INFO|
2022/10/06 12:01:37 INFO|{'vx_seed': 5, 'vir_mult': 2.38, 'booster_mult': 0, 'fit_batch': '4', 'run_type': 'fit'}: Running forward sim
2022/10/06 12:01:39 INFO|Running reports
